In [1]:
import torch
from safetensors.torch import load_file

In [2]:
ct_movq_dict = dict(torch.load("/home/mlfavorfit/.cache/huggingface/hub/models--kandinsky-community--kandinsky-2-2-controlnet-depth/snapshots/4ecd717e8c9086cf4a16ca28b64894f70a42cd08/movq/diffusion_pytorch_model.bin"))
ct_unet_dict = dict(torch.load("/home/mlfavorfit/.cache/huggingface/hub/models--kandinsky-community--kandinsky-2-2-controlnet-depth/snapshots/4ecd717e8c9086cf4a16ca28b64894f70a42cd08/unet/diffusion_pytorch_model.bin"))

un_movq_dict = dict(load_file("/home/mlfavorfit/.cache/huggingface/hub/models--kandinsky-community--kandinsky-2-2-decoder-inpaint/snapshots/db790ad5cbcabed886f069ef2710774657621702/movq/diffusion_pytorch_model.safetensors"))
un_unet_dict = dict(load_file("/home/mlfavorfit/.cache/huggingface/hub/models--kandinsky-community--kandinsky-2-2-decoder-inpaint/snapshots/db790ad5cbcabed886f069ef2710774657621702/unet/diffusion_pytorch_model.safetensors"))

In [3]:
ct_movq = sorted([(cur, ct_movq_dict[cur].shape)for cur in ct_movq_dict], key=lambda x:x[0])
ct_unet = sorted([(cur, ct_unet_dict[cur].shape)for cur in ct_unet_dict], key=lambda x:x[0])
un_movq = sorted([(cur, un_movq_dict[cur].shape)for cur in un_movq_dict], key=lambda x:x[0])
un_unet = sorted([(cur, un_unet_dict[cur].shape)for cur in un_unet_dict], key=lambda x:x[0])

In [4]:
for ct, un in zip(ct_movq, un_movq):
    if ct != un: print("warn")

In [5]:
only_ct = []
common_ct_un = []
for name, shape in ct_unet:
    if name in un_unet_dict.keys():
        common_ct_un.append(name)
    else:
        only_ct.append(name)

In [6]:
for name in common_ct_un:
    if un_unet_dict[name].shape != ct_unet_dict[name].shape: print(name)

conv_in.weight


In [7]:
un_unet_dict["conv_in.weight"].shape

torch.Size([384, 9, 3, 3])

In [8]:
ct_unet_dict["conv_in.weight"].shape

torch.Size([384, 8, 3, 3])

In [9]:
for cur in only_ct:
    print(cur, ct_unet_dict[cur].shape)

add_embedding.input_hint_block.0.bias torch.Size([16])
add_embedding.input_hint_block.0.weight torch.Size([16, 3, 3, 3])
add_embedding.input_hint_block.10.bias torch.Size([96])
add_embedding.input_hint_block.10.weight torch.Size([96, 96, 3, 3])
add_embedding.input_hint_block.12.bias torch.Size([256])
add_embedding.input_hint_block.12.weight torch.Size([256, 96, 3, 3])
add_embedding.input_hint_block.14.bias torch.Size([4])
add_embedding.input_hint_block.14.weight torch.Size([4, 256, 3, 3])
add_embedding.input_hint_block.2.bias torch.Size([16])
add_embedding.input_hint_block.2.weight torch.Size([16, 16, 3, 3])
add_embedding.input_hint_block.4.bias torch.Size([32])
add_embedding.input_hint_block.4.weight torch.Size([32, 16, 3, 3])
add_embedding.input_hint_block.6.bias torch.Size([32])
add_embedding.input_hint_block.6.weight torch.Size([32, 32, 3, 3])
add_embedding.input_hint_block.8.bias torch.Size([96])
add_embedding.input_hint_block.8.weight torch.Size([96, 32, 3, 3])


In [10]:
import torch

# 기존 텐서의 모양과 새로운 모양 정의
old_shape = (384, 9, 3, 3)
new_shape = (384, 13, 3, 3)

# 새로운 텐서 생성 및 기존 텐서 값 복사
new_weight = torch.zeros(new_shape)  # 새로운 텐서를 모두 0으로 초기화
old_weight = un_unet_dict["conv_in.weight"]
new_weight[:, :old_shape[1], :, :] = old_weight

# 추가된 부분에 랜덤한 0으로 채우기
random_part = torch.zeros((new_shape[0], new_shape[1] - old_shape[1], new_shape[2], new_shape[3]))
new_weight[:, old_shape[1]:, :, :] = random_part

# 결과 확인
print(new_weight.shape)


torch.Size([384, 13, 3, 3])


In [11]:
un_unet_dict["conv_in.weight"] = new_weight

In [12]:
for cur in only_ct:
    un_unet_dict[cur] = ct_unet_dict[cur]

In [13]:
print(len(un_unet_dict))
print(un_unet_dict["conv_in.weight"].shape)

740
torch.Size([384, 13, 3, 3])


In [15]:
from safetensors.torch import save_file

save_file(un_unet_dict, "/home/mlfavorfit/Desktop/lib_link/favorfit/kjg/0_model_weights/diffusion/Kandinsky/kandinsky-inpainting-controlnet/unet/diffusion_pytorch_model.safetensors")

In [15]:
temp_dict = load_file("/home/mlfavorfit/Desktop/lib_link/favorfit/kjg/0_model_weights/diffusion/Kandinsky/kandinsky-inpainting-controlnet/unet/diffusion_pytorch_model.safetensors")

In [16]:
print(len(temp_dict))
print(temp_dict["conv_in.weight"].shape)

740
torch.Size([384, 13, 3, 3])
